Winnie VORIHILALA <br>

# SCRAPING PYTHON DU SITE BOURSORAMA AVEC LA LIBRAIRIE REQUEST

# Chargement des librairies

In [1]:
from bs4 import BeautifulSoup as Soup, Tag
import requests
import datetime
import xlsxwriter
import pandas as pd

# Chargement des données

In [2]:
df = pd.read_excel(r'/Users/winnievorihilala/Documents/Midcap_partners/23-04-2020/DATASET_D2/D2_init.xlsx')


# Chargement des fonctions de nettoyage de textes


In [6]:
def nettoyer_nom(s): 
    s = s.replace('Raison', '')
    s = s.replace('sociale', '')
    s = s.replace('Raison sociale', '')
    s = s.replace('\n', '')
    s = s.replace('  ', '')
    return(s)
def nettoyer_adresse(s): 
    #s = s.replace('\n', '')
    #s = s.replace('  ', '')
    if 'Norvège' in s :
        s = 'Norvège'
    elif 'France' in s :
        s = 'France'
    elif 'Suisse' in s :
        s = 'Suisse'
    elif 'London' in s :
        s = 'UK'
    elif 'Espagne' in s :
        s = 'Espagne'
    elif 'Italie' in s :
        s = 'Italie'
    elif 'Belgique' in s :
        s = 'Belgique'
    elif 'Allemagne' in s :
        s = 'Allemagne'
    elif 'Danemark' in s :
        s = 'Danemark'
    elif 'Pays-Bas' in s :
        s = 'Pays-Bas'
    elif 'Suède' in s :
        s = 'Suède'
    elif 'Luxembourg' in s :
        s = 'Luxembourg'
    elif 'Royaume-Uni' in s :
        s = 'UK'
    elif 'Irlande' in s :
        s = 'Irlande'
    elif 'États-Unis' in s :
        s = 'États-Unis'
    return(s)
def nettoyer_description(s): 
    s = s.replace('\n', '')
    return(s)
def nettoyer_marketcap(s): 
    s = s.replace('Capitalisation', '')
    s = s.replace('boursière', '')
    s = s.replace('Capitalisation boursière', '')
    s = s.replace('\n', '')
    s = s.replace('  ', '')
    return(s)
def nettoyer_marche(s): 
    s = s.replace('Marché', '')
    s = s.replace('\n', '')
    s = s.replace('  ', '')
    return(s)
def nettoyer_titres(s): 
    s = s.replace('Nombre de titres', '')
    s = s.replace('\n', '')
    s = s.replace('  ', '')
    return(s)
def nettoyer_site(s): 
    s = s.replace('Site web', '')
    s = s.replace('Site', '')
    s = s.replace('Web', '')
    s = s.replace('\n', '')
    s = s.replace('  ', '')
    return(s)

# Récupération des données

In [7]:
data = []

for i in range(0,df.shape[0]-1):
    response = requests.get(df.iloc[i,0])
    soup = Soup(response.content)
    try:
        description = str(soup.find('p', attrs = {'class':'o-ellipsis-multiline-6'}).text).strip()
    except:
        description = 'None'   
    form_div = soup.find("div", {"class": "c-list-info c-list-info--no-gutter@md-min"})
    maliste = []
    for item in form_div.findAll('li'):
        if isinstance(item, Tag):
            #print(item.text)
            maliste.append(item.text)
    nom = maliste[0]
    siege_social = maliste[1]
    site_web = maliste[2]
    titres = maliste[5]
    marketcap = maliste[6]
    marche = maliste[7]
    
    nom = nettoyer_nom(nom)
    siege_social = nettoyer_adresse(siege_social)
    description = nettoyer_description(description)
    marketcap = nettoyer_marketcap(marketcap)
    marche = nettoyer_marche(marche)
    titres = nettoyer_titres(titres)
    site_web = nettoyer_site(site_web)

    date=datetime.datetime.today().strftime('%d-%m-%Y')
    data.append({'Date':date,'Nom':nom,'Siège social':siege_social,'Description':description,'Capitalisation boursière':marketcap,'Marché':marche,'Nombre de titres':titres,'Site web':site_web})



# Export sous format excel

In [8]:
workbook = xlsxwriter.Workbook(r'/Users/winnievorihilala/Documents/Midcap_partners/23-04-2020/DATASET_D2/Coverage_Midcap_Partners.xlsx')
worksheet = workbook.add_worksheet('Coverage')
bold = workbook.add_format({'bold': True})
worksheet.write(0,0,'Date',bold)
worksheet.write(0,1,'Nom',bold)
worksheet.write(0,2,'Siège social',bold)
worksheet.write(0,3,'Description',bold)
worksheet.write(0,4,'Capitalisation boursière',bold)
worksheet.write(0,5,'Marché',bold)
worksheet.write(0,6,'Nombre de titres',bold)
worksheet.write(0,7,'Site web',bold)

for i in range(1,len(data)+1):
    try:
        worksheet.write(i,0,data[i-1]['Date'])
    except:
        pass
    try:
        worksheet.write(i,1,data[i-1]['Nom'])
    except:
        pass
    try:
        worksheet.write(i,2,data[i-1]['Siège social'])
    except:
        pass
    try:
        worksheet.write(i,3,data[i-1]['Description'])
    except:
        pass
    try:
        worksheet.write(i,4,data[i-1]['Capitalisation boursière'])
    except:
        pass
    try:
        worksheet.write(i,5,data[i-1]['Marché'])
    except:
        pass
    try:
        worksheet.write(i,6,data[i-1]['Nombre de titres'])
    except:
        pass
    try:
        worksheet.write(i,7,data[i-1]['Site web'])
    except:
        pass
workbook.close()